# Goalkeepr OLS
- Import Package
- Connect DB & get Forword Player Data
- Scaling
- Summary OLS 
- Remove Feature
- Anova & Remove Feature

### Import Package

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import MySQLdb

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

### Connect DB & get Forword Player Data

In [12]:
db = MySQLdb.connect(
    "db.fastcamp.us",
    "root",
    "dkstncks",
    "football",
    charset='utf8',
)

def make_query(position):
    """
    
    parameter------------
    position : M, D, F, G
    
    return---------------
    SQL_QUERY String
    
    """
    SQL_QUERY = """
        SELECT 
            age, tall, weight, apps_start, apps_sub, mins, goals, assists, yel, red
            , spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb, blocks
            , owng, keyp_x, fouled, off, disp, unstch, avgp, ps_y, rating
        FROM player
        WHERE position like "%{position}%"
        ;
    """.format(position=position)
    
    return SQL_QUERY

# forword
SQL_QUERY = make_query("G")
goalkeeper_df = pd.read_sql(SQL_QUERY, db)

len(goalkeeper_df)

289

##### Scaling

In [13]:
X = goalkeeper_df.ix[:,:-1]
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)

dfX0 = pd.DataFrame(X_scaled, columns=X.columns)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(goalkeeper_df.ix[:,-1], columns=["rating"])
g_df = pd.concat([dfX, dfy], axis=1)
g_df.head()

,age,tall,weight,apps_start,apps_sub,mins,goals,assists,yel,red,...,blocks,owng,keyp_x,fouled,off,disp,unstch,avgp,ps_y,rating
0,7.432029,15.948156,11.867021,2.703886,0.000000,2.646419,0.0,0.0,0.816273,3.232331,...,0.0,0.0,1.329329,0.678815,0.0,0.0,0.000000,4.708980,3.971232,6.62
1,5.945623,15.614164,12.139826,0.073078,2.104282,0.141947,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,5.616214,3.558452,5.80
2,4.883905,15.697662,11.321411,2.776964,0.000000,2.789998,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.678815,0.0,0.0,1.120947,2.692903,3.316477,6.83
3,7.007342,15.363669,10.912203,2.338496,0.000000,2.347841,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,3.844947,5.999550,6.92
4,5.096249,15.614164,11.594216,0.438468,2.104282,0.442158,0.0,0.0,0.000000,0.000000,...,0.0,0.0,1.329329,0.000000,0.0,0.0,0.000000,3.110519,5.935498,6.48


##### Summary OLS

In [14]:
model = sm.OLS(g_df.ix[:, -1], g_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     2027.
Date:                Sat, 25 Jun 2016   Prob (F-statistic):          5.85e-286
Time:                        19:20:29   Log-Likelihood:                -197.13
No. Observations:                 289   AIC:                             444.3
Df Residuals:                     264   BIC:                             535.9
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age            0.0893      0.031      2.919      0.0

##### Remove Some Feature

In [17]:
# remove features
remove_column_list = [
    "weight", "apps_start", "apps_sub", "mins", "goals", "assists", "yel", "red", "spg", "motm"
    , "tackles","inter", "fouls", "blocks", "owng", "keyp_x", "fouled", "off", "disp", "unstch"
]
removed_g_df = g_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed_g_df.ix[:, -1], removed_g_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     7263.
Date:                Sat, 25 Jun 2016   Prob (F-statistic):          2.67e-314
Time:                        19:24:03   Log-Likelihood:                -206.16
No. Observations:                 289   AIC:                             426.3
Df Residuals:                     282   BIC:                             452.0
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age            0.0814      0.029      2.810      0.0

##### Anova &  Remove Feature

In [19]:
formula_str = """
rating ~ age + tall + ps_x + aw + offsides + clear + drb +  avgp + ps_y
"""

model = sm.OLS.from_formula(formula_str, data=removed_g_df)
result = model.fit()
table_anova = sm.stats.anova_lm(result)
table_anova

,df,sum_sq,mean_sq,F,PR(>F)
age,1.0,0.000003,0.000003,0.000022,0.996227
tall,1.0,0.234744,0.234744,1.853932,0.174417
ps_x,1.0,0.998707,0.998707,7.887450,0.005327
aw,1.0,1.994044,1.994044,15.748282,0.000092
offsides,1.0,0.036328,0.036328,0.286909,0.592632
clear,1.0,1.943786,1.943786,15.351360,0.000112
drb,1.0,0.776576,0.776576,6.133133,0.013856
avgp,1.0,0.973691,0.973691,7.689884,0.005925
ps_y,1.0,0.001614,0.001614,0.012749,0.910181
Residual,281.0,35.580157,0.126620,NaN,NaN


In [20]:
# remove feature 2
remove_column_list = [
    "age", "offsides", "ps_y"
]
removed2_g_df = removed_g_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed2_g_df.ix[:, -1], removed2_g_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     8271.
Date:                Sat, 25 Jun 2016   Prob (F-statistic):          1.38e-314
Time:                        19:25:26   Log-Likelihood:                -210.15
No. Observations:                 289   AIC:                             432.3
Df Residuals:                     283   BIC:                             454.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
tall           0.3411      0.009     37.102      0.0